In [1]:
import numpy as np

from deeprte.utils import flat_dict_to_rte_data
from deeprte.model.config import CONFIG
from deeprte.model.rte import RTEOperator
from deeprte.model.data import flat_params_to_haiku
from deeprte.model.modules import FunctionInputs


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PARAMS_FILE = "../data/experiments/square_full_1_2022-02-09T00:04:32/models/latest/step_500000_2022-02-10T08:51:07/params.npz"
DATA_FILE = "../data/train/square_full_1.npz"

np_params = np.load(PARAMS_FILE)
params = flat_params_to_haiku(np_params)

np_data = np.load(DATA_FILE)
rte_data = flat_dict_to_rte_data(np_data)
data, grid = rte_data["data"], rte_data["grid"]

sigma_t, sigma_a, psi_raw = data["sigma_t"], data["sigma_a"], data["psi_bc"]
sigma_raw = np.stack([sigma_a, sigma_t], axis=-1)
r, v, w_angle, rv_prime, w_prime = grid["r"], grid["v"], grid["w_angle"], grid["rv_prime"], grid["w_prime"]

In [3]:
solution = RTEOperator(CONFIG.rte_operator, use_jit=True)

In [6]:
sigma = FunctionInputs(x=r.reshape(-1, 2), f=sigma_raw[0:1].reshape(1, -1, 2))
psi_bc = FunctionInputs(x=rv_prime.reshape(-1, 4), f=psi_raw[0:1].reshape(1, -1) * w_prime.flatten())
out, state = solution.vrho(params, None, 0, r.reshape(-1, 2), sigma, psi_bc, [v, w_angle])